In [ ]:
#Import general libraries (needed for functions)
import numpy as np
#import matplotlib.pyplot as plt
from IPython import display

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

#Import Qiskit classes 
import qiskit
import time
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import __version__
from qiskit.quantum_info import random_statevector
from numpy import random
from qiskit import QuantumCircuit

# random is needed for the initial X gates. In order to minimise SPAM (i.e. B=0.5/0.25 in old parlance)
# a random mix of which state we return it to is best. Could seed this if you want.

import numpy.random as rand
from qiskit import Aer, execute
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit.tools.visualization import plot_histogram, plot_state_city

# The rest are for saving and logging and list flattening.
import os
import pickle
from functools import reduce
from itertools import chain
import datetime

In [ ]:
binnQ = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(nQ)] ) )
nQ = 5

In [ ]:
# Make the data directory if it doesn't exist 
try: 
    os.makedirs("data")
except:
    print()
# Make the temp save directory if it doesn't exist    
try: 
    os.makedirs("temp")
except:
    print()

In [ ]:
# If you don't specify an initial layout MAKE SURE the qiskit transpiler doesn't mix and match the qubit location.
def generateInitialLayout(q):
  """ Pass in the qregister and generate a one to one initial layout """
  initial_layout = {}
  for (idx,q) in enumerate(q):
    initial_layout[q] = idx
  return initial_layout

In [ ]:
#run rb_circs for each string (2^nQ)
def createInitializeCircuits(rb_pattern,nseeds,lengths,nQ,start=0,end=2**nQ-1):
    rb_opts = {}
    rb_opts['length_vector'] = lengths
    rb_opts['nseeds'] = (end-start+1)*nseeds
    rb_opts['rb_pattern'] = rb_pattern
    rb_circs, xdata = rb.randomized_benchmarking_seq(**rb_opts)
    
    for n in range(start,end+1):
        for i in range(nseeds):
            for j in range(len(lengths)):
                    qc=QuantumCircuit(nQ,nQ)
                    binary=binnQ(n)

                    for k in range(nQ):
                        if binary[k]=='1':
                            qc.x(nQ-1-k)
                    qc.barrier()
                    rb_circs[(n-start)*nseeds+i][j]=qc.compose(rb_circs[(n-start)*nseeds+i][j])
    
    return (rb_circs)

In [ ]:
def countQueuedJobs(jobs):
    """
    Checks how many jobs are queued, running or about to be queued
    The 'fair use' policy appears to only allow you to stack the queue to a certain amount
    I think different 'rights' allow different numbers to be queued..
    """
    count = 0
    for j in jobs:
        job_status = j.status()
        if job_status == JobStatus.QUEUED or \
           job_status == JobStatus.INITIALIZING or \
           job_status == JobStatus.VALIDATING or \
           job_status == JobStatus.RUNNING:
            count += 1
    return count

In [ ]:
from qiskit.providers import JobStatus


def saveJobs(js,savePrefix,forced = False):
    """
    savePairs(bs,js,start,savePrefix,forced = False)
    
    Designed to save jobs as and when the results become available.
    We don't want to block, however, rather we will iterate through a list
    and try to deal gracefully with jobs that are not ready or that
    give us an error when we try and retrieve them.
    If the file is there, the job will be skipped UNLESS forced is true.
    As long as we have the job_id (which should have been saved seperately)
    we should be able to retrieve them later anyway.
    bs: the bitstrings associated with the jobs
    js: a list of the jobs we have
    start: the number the jobs start at
    savedPrefix: What do we want the saved pickles to start with.
    forced: Set it to true if you want to save the job, even if we think it has been saved before.
    """
    for (idx,j) in enumerate(js):
        saved_this_one = False
        if forced or not os.path.isfile(savePrefix+str(idx)+'.pickle'):
           try:
              if j.status() == JobStatus.DONE:
                 res = j.result()
                 with open(savePrefix+str(idx)+'.pickle','wb') as f:
                        pickle.dump(res,f)
                 saved_this_one = True
           except ApiError as ex:
              print("There was a api exception error")
              template = "An exception of type {0}. Args \n{1!r}"
              message = template.format(type(ex).__name__,ex.args)
              print(message)
              print("Ignoring this for now, but NOT saved.")
           except Exception as ex2:
              print("There was a non-api exception error")
              template = "An exception of type {0}. Args \n{1!r}"
              message = template.format(type(ex2).__name__,ex2.args)
              print(message)
              print("Ignoring this for now, but NOT saved.")
           if saved_this_one == True:
              now = datetime.datetime.now()
              print("Saved: "+ savePrefix[5:] + str(idx)+": " + now.strftime("%d %B: %r "))

In [ ]:
def divideCircuits(flat_circuits,backend):
    """
    This function divides the flat_circuits in accordance
    with the maximum number of circuits for the backend
    The returned circuits are arrays of the form
    (len(flat_circuits)/max_experiments,max_experiments)
    """
    max_experiments=backend.configuration().max_experiments
    #The total number of jobs is the len(flat_circuits) divided by the max_experiments per job
    jobs_number = int(np.floor(len(flat_circuits)/max_experiments))
    
    circuits=[]
    for idx in range(jobs_number):
        ls = flat_circuits[max_experiments*idx:max_experiments*(idx+1)]
        circuits.append(ls)

    if jobs_number*max_experiments<len(flat_circuits):
        ls = flat_circuits[max_experiments*jobs_number:]
        circuits.append(ls)

    return circuits

In [ ]:
def send(circuits,initial_layout,backend,shots=1024):
    """
      backend: The backend you want
      shots: the number of shots to take.
      
      returns: job created and sent.
    """
    # This will not be idiomatic Python - should find out `better` way to write this
    # Basically if we don't get an error free response from a backend query, sleep and try again.
    max_credits=5
    errorFree = False
    coupling_map = backend.configuration().coupling_map
    # If its down wait for it to come back up. 
    # This attempts to stop everything going horribly wrong cause the machine is down for e.g. maintenance.
    while not errorFree:
        try:
            while not backend.status().operational:
                now = datetime.datetime.now()
                print("Device not operational at", now.strftime("%d %B: %r"), ", sleeping for a bit")
                time.sleep(60*10) # 10 minutes.
            errorFree = True
        except KeyboardInterrupt:
            print("Keboard interrupt whilst waiting for the backend to come back up")
            print("Going to return with what has been done so far")
            return (jobsList,bitFlips)
        except Exception as ex:
            print("Got an exception whilst checking the backend operational status. It is almost certainly just a maintenance issue.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex).__name__,ex.args)
            print(message)
            print("Ill sleep for 10 minutes and then recheck.")
            time.sleep(60*10)
    # if we run with simulator take out the coupling.
    errorFree = False
    while not errorFree:
        try:
            job_exp = execute(circuits, backend=backend, shots=shots, max_credits=max_credits,initial_layout=initial_layout,coupling_map=coupling_map,memory=True)
            errorFree=True
        except Exception as ex:
            print("Got an exception whilst trying to submit the job. Probably a too many concurrent etc.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex).__name__,ex.args)
            print(message)
            print("Ill sleep for 3 minutes and then recheck.")
            time.sleep(60*3)
    return job_exp
    

In [ ]:
def getAndSaveJobID(job_exp,index,id_save_name="temp/JobID_temp_"):
    """
        Given a list of bits (x gates applied) and a list of jobs (returned from execute)
        We retrieve the job_id and then save it with the corresponding bitpattern of x gates.
        in a pickle starting at index and using a prefix
        of id_save_name (default JobIDandBit_temp_)
        
        The purpose of this is incase something goes wrong before the
        job itself can be saved - if we have the bits and the id we can
        always get the job results later.
    """
    # So now we want the jobid so we can pickle it. Can't pickle the job_exp as its threadlocked.
    # We can get errors trying to get the jobid from the submitted job
    # Try to deal with them gracefully.
    try:
        jobId = job_exp.job_id()
        with open(id_save_name+str(index)+".pickle",'wb') as f:
            pickle.dump(jobId,f)
        now = datetime.datetime.now()
        print("Saved jobid: ",index,":", now.strftime("%d %B: %r"))
    except Exception as ex:
        template = "An exception of type {0}. Args \n{1!r}"
        message = template.format(type(ex).__name__,ex.args)
        print(message)
        print("We got an error, probably timing, things might be busy job ids not available. etc")
        time.sleep(60*2)
        try:
            jobId = job_exp.job_id()
            with open(id_save_name+str(index)+".pickle",'wb') as f:
                pickle.dump(jobId,f)
            now = datetime.datetime.now()
            print("Saved jobid: ",index,": ", now.strftime("%d %B: %r"))
        except Exception as ex2:
            print("That really didn't work, so just NOT SAVING, well only the bits. You will have the job in the returned lists, lose that an you have to re-run this.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex2).__name__,ex2.args)
            print(message)
    return jobId


In [ ]:
def flattenCircuits(rb_circs_initialized,m):
    flat_circuits=[]
    for i in range(len(rb_circs_initialized)):
        flat_circuits.append(rb_circs_initialized[i][m])
    return flat_circuits

In [ ]:
def sendAJobWhenOk(rb_circs_initialized,nQ,lengths,backend,maxQueue = 5,shots=1024,saveFile="data/Melbourne15_"):
    """
    sendAJobWhenOk(nQ,lengths,nseeds,rb_pattern,backendToUse,start,end,maxQueue = 3,saveFile="Melbourne15")
    Sends off a number of jobs (from start to end)
    Only allows queue size to be 3 (you may want to alter if you can have more in queue)
    Saves numbered with saveFile as a prefix.
    You also need to supply:
      nQ, the number of qubits
      lengths, the sequence lenths as a list
      nseeds, the number of experiments in a single batch - qasm limits will limit this depending on lengths
      backendToUse, err the backend you want to use
      start, start numbering here
      end, end numbering here (well one before here I suppose).
      shots: (default = 1024) number of shots per circuit
    """
    
    initial_layout=generateInitialLayout(rb_circs_initialized[0][0].qregs[0])
    
    all_jobs = []
    all_job_ids = []
    
    for m in range(len(lengths)):
        m_jobs = []
        m_job_ids = []
        
        flat_circuits = flattenCircuits(rb_circs_initialized,m)
        circuits = divideCircuits(flat_circuits,backend)
        
        for toDo in range(len(circuits)):
            good_to_go = False
            while not good_to_go:
                counted = countQueuedJobs(m_jobs)
                if counted >= maxQueue:
                    print("We have {} jobs in queue, sleeping: {}".format(counted,datetime.datetime.now()))
                    time.sleep(60*2)
                    
                    saveJobs(m_jobs,saveFile+"m_"+str(m)+"_job_",forced = False)
                else:
                    good_to_go = True

            job_exp = send(circuits[toDo],initial_layout,backend,shots=shots)
            job_exp.update_name("m_"+str(m)+"_job_"+str(toDo))
            job_id = getAndSaveJobID(job_exp,"m_"+str(m)+"_job_"+str(toDo))
            m_jobs.append(job_exp)
            m_job_ids.append(job_id)
    
        counted = countQueuedJobs(m_jobs)
        while counted > 0:
            print("Still have {} jobs running: {}. Waiting for 120 seconds.".format(counted,datetime.datetime.now()))
            time.sleep(2*60)
            counted = countQueuedJobs(m_jobs)
        saveJobs(m_jobs,saveFile+"m_"+str(m)+"_job_",forced = False)
    
        all_jobs.append(m_jobs)
        all_job_ids.append(m_job_ids)
            
    return (all_jobs,all_job_ids)


In [ ]:
provider = qiskit.IBMQ.load_account()
backend = provider.get_backend("ibmq_belem")

In [ ]:
nQ=5
nseeds = 1000
lengths= [1,10,20,30,40,50,60,70,80,100]
start=0
end=31
rb_pattern = [[0],[1],[2],[3],[4]]

In [ ]:
rb_circs = createInitializeCircuits(rb_pattern=rb_pattern,
                                    nseeds=nseeds,
                                    lengths=lengths,
                                    nQ=nQ,
                                    start=start,
                                    end=end)

In [ ]:
all_jobs,all_job_ids=sendAJobWhenOk(rb_circs_initialized=rb_circs,
                                    nQ=nQ,
                                    lengths=lengths,
                                    backend=backend,
                                    maxQueue = 5,
                                    shots=1024,
                                    saveFile="data/Belem_")